### GraphFlow

In [21]:
import asyncio
from autogen_agentchat.agents import AssistantAgent,UserProxyAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination
from dotenv import load_dotenv
import os

# Load API key
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Model client
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

### DigraphBuilder GraphFlow

In [22]:
from autogen_agentchat.teams import DiGraphBuilder, GraphFlow

### Sequential Flow

In [23]:
writer = AssistantAgent(
    name="Writer",
    description="A writer agent that generates text based on user input.",
    model_client=model_client,
    system_message="You are a creative writer. Please write a story based on the user's input.",
)

reviewer = AssistantAgent(
    name="Reviewer",
    description="A reviewer agent that provides feedback on the text generated by the writer.",
    model_client=model_client,
    system_message="You are a reviewer. Please provide feedback on the text generated by the writer.",
)

In [24]:
builder = DiGraphBuilder()
builder.add_node(writer).add_node(reviewer)
builder.add_edge(writer, reviewer)

graph = builder.build()

In [25]:
graph

DiGraph(nodes={'Writer': DiGraphNode(name='Writer', edges=[DiGraphEdge(target='Reviewer', condition=None, condition_function=None, activation_group='Reviewer', activation_condition='all')], activation='all'), 'Reviewer': DiGraphNode(name='Reviewer', edges=[], activation='all')}, default_start_node=None)

In [26]:
team = GraphFlow([writer,reviewer], graph)


In [27]:
stream = team.run_stream(task ="Write a good poem about India in less than 30 words.")

async for event in stream:
    print(event)

id='d6d390f8-c4fe-43c2-ad55-1e75db8ca54f' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 19, 7, 8, 20, 312432, tzinfo=datetime.timezone.utc) content='Write a good poem about India in less than 30 words.' type='TextMessage'
id='d1947567-9fcd-48ef-9cef-01c8900c03ea' source='Writer' models_usage=RequestUsage(prompt_tokens=41, completion_tokens=40) metadata={} created_at=datetime.datetime(2025, 7, 19, 7, 8, 22, 863724, tzinfo=datetime.timezone.utc) content="Land of vibrant hues and ancient lore,  \nUnity in diversity, cherished evermore.  \nFrom Himalayas' majesty to Kerala's shores,  \nIndia, a timeless tapestry, forever explores.  " type='TextMessage'
id='af1d5db2-8f37-4f42-93e6-99de629d0f6c' source='Reviewer' models_usage=RequestUsage(prompt_tokens=87, completion_tokens=148) metadata={} created_at=datetime.datetime(2025, 7, 19, 7, 8, 25, 779437, tzinfo=datetime.timezone.utc) content='This is a concise and evocative poem that effectively captures India\

In [28]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.teams import DiGraphBuilder, GraphFlow
from autogen_ext.models.openai import OpenAIChatCompletionClient


async def main():
    # Initialize agents with OpenAI model clients.
    model_client = OpenAIChatCompletionClient(model="gpt-4.1-nano")
    agent_a = AssistantAgent("A", model_client=model_client, system_message="You are a helpful assistant.")
    agent_b = AssistantAgent("B", model_client=model_client, system_message="Translate input to Chinese.")
    agent_c = AssistantAgent("C", model_client=model_client, system_message="Translate input to Japanese.")

    # Create a directed graph with fan-out flow A -> (B, C).
    builder = DiGraphBuilder()
    builder.add_node(agent_a).add_node(agent_b).add_node(agent_c)
    builder.add_edge(agent_a, agent_b).add_edge(agent_a, agent_c)
    graph = builder.build()

    # Create a GraphFlow team with the directed graph.
    team = GraphFlow(
        participants=[agent_a, agent_b, agent_c],
        graph=graph,
        termination_condition=MaxMessageTermination(5),
    )

    # Run the team and print the events.
    async for event in team.run_stream(task="Write a short story about a cat."):
        print(event)


asyncio.run(main())

id='735dfe6b-bb7e-4640-b3d7-350988f9654c' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 19, 7, 8, 25, 848062, tzinfo=datetime.timezone.utc) content='Write a short story about a cat.' type='TextMessage'
id='16da2324-6b53-4481-99a5-7c47c09d4cc3' source='A' models_usage=RequestUsage(prompt_tokens=26, completion_tokens=225) metadata={} created_at=datetime.datetime(2025, 7, 19, 7, 8, 27, 720971, tzinfo=datetime.timezone.utc) content='Once upon a time, in a cozy little village, there was a curious cat named Whiskers. With shiny gray fur and sparkling green eyes, Whiskers loved exploring every nook and cranny of the neighborhood. One sunny morning, he decided to venture into the woods beyond the bakery.\n\nAs he padded softly through the tall grass, Whiskers discovered a tiny, shimmering pond. To his surprise, a family of ducks was swimming happily nearby. Whiskers watched quietly, fascinated by their gentle quacks and playful splashes.\n\nSuddenly, a flutt

In [29]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.teams import DiGraphBuilder, GraphFlow
from autogen_ext.models.openai import OpenAIChatCompletionClient



    # Initialize agents with OpenAI model clients.
model_client = OpenAIChatCompletionClient(model="gpt-4.1-nano")
agent_a = AssistantAgent("A", model_client=model_client, system_message="You are a helpful assistant.")
agent_b = AssistantAgent("B", model_client=model_client, system_message="Translate input to Chinese.")
agent_c = AssistantAgent("C", model_client=model_client, system_message="Translate input to Japanese.")
agent_d = AssistantAgent("D", model_client=model_client, system_message="Summarize.")


# Create a directed graph with fan-out flow A -> (B, C).
builder = DiGraphBuilder()
builder.add_node(agent_a).add_node(agent_b).add_node(agent_c).add_node(agent_d)
builder.add_edge(agent_a, agent_b).add_edge(agent_a, agent_c)
builder.add_edge(agent_b, agent_d).add_edge(agent_c, agent_d)
graph = builder.build()

# Create a GraphFlow team with the directed graph.
team = GraphFlow(
    participants=[agent_a, agent_b, agent_c, agent_d],
    graph=graph,
    termination_condition=MaxMessageTermination(5),
)

# Run the team and print the events.
async for event in team.run_stream(task="Write a short story about a cat."):
    print(event)

id='148ae690-043a-45b1-8f0b-837ee0ac3fce' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 19, 7, 8, 30, 689762, tzinfo=datetime.timezone.utc) content='Write a short story about a cat.' type='TextMessage'
id='394c7e53-3352-42a0-a373-62cf48555b1d' source='A' models_usage=RequestUsage(prompt_tokens=26, completion_tokens=314) metadata={} created_at=datetime.datetime(2025, 7, 19, 7, 8, 32, 405200, tzinfo=datetime.timezone.utc) content="Once upon a time, in a cozy little village, lived a curious cat named Whiskers. With sleek gray fur and bright green eyes, Whiskers loved exploring every nook and cranny of his neighborhood. Each morning, he would leap out of his basket, stretch luxuriously, and set off on a new adventure.\n\nOne sunny day, Whiskers discovered a mysterious, sparkling object beneath an ancient oak tree. It was a tiny, shimmering key. Intrigued, he padded over to his favorite spot — a abandoned garden filled with colorful flowers and rusted sta

In [30]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.teams import DiGraphBuilder, GraphFlow
from autogen_ext.models.openai import OpenAIChatCompletionClient


async def main():
    # Initialize agents with OpenAI model clients.
    model_client = OpenAIChatCompletionClient(model="gpt-4.1-nano")
    agent_a = AssistantAgent(
        "A",
        model_client=model_client,
        system_message="Detect if the input is in Chinese. If it is, say 'yes', else say 'no', and nothing else.",
    )
    agent_b = AssistantAgent("B", model_client=model_client, system_message="Translate input to English.")
    agent_c = AssistantAgent("C", model_client=model_client, system_message="Translate input to Chinese.")

    # Create a directed graph with conditional branching flow A -> B ("yes"), A -> C (otherwise).
    builder = DiGraphBuilder()
    builder.add_node(agent_a).add_node(agent_b).add_node(agent_c)
    # Create conditions as callables that check the message content.
    builder.add_edge(agent_a, agent_b, condition=lambda msg: "yes" in msg.to_model_text())
    builder.add_edge(agent_a, agent_c, condition=lambda msg: "yes" not in msg.to_model_text())
    graph = builder.build()

    # Create a GraphFlow team with the directed graph.
    team = GraphFlow(
        participants=[agent_a, agent_b, agent_c],
        graph=graph,
        termination_condition=MaxMessageTermination(5),
    )

    # Run the team and print the events.
    async for event in team.run_stream(task="AutoGen is a framework for building AI agents."):
        print(event)


asyncio.run(main())

id='baddad9d-6413-48e5-8a34-12ac1e70ee35' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 19, 7, 8, 41, 372573, tzinfo=datetime.timezone.utc) content='AutoGen is a framework for building AI agents.' type='TextMessage'
id='6571f1b2-c7a2-4f26-9365-241194a015aa' source='A' models_usage=RequestUsage(prompt_tokens=47, completion_tokens=1) metadata={} created_at=datetime.datetime(2025, 7, 19, 7, 8, 41, 875061, tzinfo=datetime.timezone.utc) content='no' type='TextMessage'
id='3482b81b-fc7f-4747-82d4-df0a51082c27' source='C' models_usage=RequestUsage(prompt_tokens=33, completion_tokens=14) metadata={} created_at=datetime.datetime(2025, 7, 19, 7, 8, 42, 292622, tzinfo=datetime.timezone.utc) content='AutoGen 是一个用于构建人工智能代理的框架。' type='TextMessage'
messages=[TextMessage(id='baddad9d-6413-48e5-8a34-12ac1e70ee35', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 19, 7, 8, 41, 372573, tzinfo=datetime.timezone.utc), content='AutoGen 